In [ ]:
from kafka import KafkaClient, KafkaConsumer
from cassandra import ConsistencyLevel
from cassandra.cluster import Cluster
from cassandra.query import SimpleStatement
import threading, logging, time
import avro.schema
import avro.io
import io

KEYSPACE = "slack_pack"

# Connect to the default (localhost) cluster
cluster = Cluster()
session = cluster.connect()

# Create keyspace
session.execute("""
        CREATE KEYSPACE IF NOT EXISTS %s
        WITH replication = { 'class': 'SimpleStrategy', 'replication_factor': '2' }
        """ % KEYSPACE)

session.set_keyspace(KEYSPACE)

# Create Table
session.execute("""
        CREATE TABLE IF NOT EXISTS awaybot (
            rec_id text,
            user_id text,
            message_text text,
            channel_id text,
            team_id text,
            ts text,
            PRIMARY KEY (rec_id)
        )
        """)

# Write pre-prepared insert statement
prepared = session.prepare("""
        INSERT INTO awaybot (rec_id, user_id, message_text,
            channel_id, team_id, ts)
        VALUES (?, ?, ?, ?, ?, ?)
        """)

# Create a kafka consumer
consumer = KafkaConsumer('T2BT8MVE3')
# Read in the avro schema
schema = avro.schema.parse(open("slack.avsc", "rb").read())


c = 0
# Consume messages
for msg in consumer:
    c += 1
    if not c % 100:
        print c
    bytes_reader = io.BytesIO(msg.value)
    decoder = avro.io.BinaryDecoder(bytes_reader)
    reader = avro.io.DatumReader(schema)
    message = reader.read(decoder)
    # Insert each message into Cassandra
    session.execute(
        prepared, 
        (message['rec_id'], message['user_id'],
         message['message_text'], message['channel_id'], 
         message['team_id'], message['ts']))


100
200
300
400
500
600
700
800
900
1000
1100
1200
